# <font color='black'>**Kickstarter Prediction Challenge - Feature Engineering Competition**</font>

## </font='black'>This is my approach for a feature engineering competition at HackerEarth https://www.hackerearth.com/problem/machine-learning/funding-successful-projects/. This got us 23rd place on the final leaderboard among ~500 particpants. The dataset is available for download at the above link.</font>

In [5]:
import pandas as pd
import numpy as np
import numpy as np
from sklearn import svm
import xgboost as xgb
from xgboost import XGBClassifier
from currency_converter import CurrencyConverter
from sklearn import metrics
import time
from sklearn.cross_validation import train_test_split
import xgboost
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
c=CurrencyConverter()


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print ('The train data has {} rows and {} columns'.format(train.shape[0],train.shape[1]))
print ('The test data has {} rows and {} columns'.format(test.shape[0],test.shape[1]))


The train data has 108129 rows and 14 columns
The test data has 63465 rows and 12 columns


# </font='black'> Let us take a look at the features available to engineer further </font>

In [4]:
train.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,1241333999,1241334017,1240600507,1240602723,3,1
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,1242429000,1242432018,1240960224,1240975592,2,0
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,1243027560,1243027818,1242163613,1242164398,0,0
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,False,US,USD,1243555740,1243556121,1240963795,1240966730,18,1
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,1243769880,1243770317,1241177914,1241180541,1,0


## </font='black'>The dataset contains fields like - Name of the project, Description, The goal amount etc as seen above. We will extract text features as well as normalized time features from the given variables. First we extract time features like duration of project, time project changed after launched.</font>

In [3]:
train['time_project']=train['deadline']-train['launched_at']
train['time_deadline_change']=train['state_changed_at']-train['deadline']
train['time_launched_change']=train['state_changed_at']-train['launched_at']
train['time_launched_change']=train['time_launched_change']/1e4
train['time_created_change']=train['state_changed_at']-train['created_at']
train['time_created_change']=train['time_created_change']/1e4
train['time_project']=train['time_project']/1e4
train['cldiff']=train['created_at']-train['launched_at']
train['cldiff']=train['cldiff']/1e3
train['time_deadline_change_log']=train['time_deadline_change'].apply(lambda c: np.log(c))
train['time_launched_change_log']=train['time_launched_change'].apply(lambda c: np.log(c))
train['time_created_change_log']=train['time_created_change'].apply(lambda c: np.log(c))
train['time_project_log']=train['time_project'].apply(lambda c: np.log(c))
train['year'] = train['deadline'].apply(lambda k: int(time.ctime(k)[-4:])-2009)

In [4]:
test['year'] = test['deadline'].apply(lambda k: int(time.ctime(k)[-4:])-2009)

test['time_project']=test['deadline']-test['launched_at']
test['time_deadline_change']=test['state_changed_at']-test['deadline']
test['time_launched_change']=test['state_changed_at']-test['launched_at']
test['time_launched_change']=test['time_launched_change']/1e4
test['time_project']=test['time_project']/1e4
test['time_created_change']=test['state_changed_at']-test['created_at']
test['time_created_change']=test['time_created_change']/1e4
test['cldiff']=test['created_at']-test['launched_at']
test['time_deadline_change_log']=test['time_deadline_change'].apply(lambda c: np.log(c))
test['time_launched_change_log']=test['time_launched_change'].apply(lambda c: np.log(c))
test['time_created_change_log']=test['time_created_change'].apply(lambda c: np.log(c))
test['time_project_log']=test['time_project'].apply(lambda c: np.log(c))
test['cldiff']=test['cldiff']/1e3

In [5]:
train['month'] = train['deadline'].apply(lambda k: time.ctime(k).split()[1])
test['month'] = test['deadline'].apply(lambda k: time.ctime(k).split()[1])



## </font='black'>We now compute features like goal per unit description length, normalized project goal amount etc. </font>

In [6]:
cols_to_use = ['name','desc']
len_feats = ['name_len','desc_len']
count_feats = ['name_count','desc_count']

for i in np.arange(2):
    train[len_feats[i]] = train[cols_to_use[i]].apply(str).apply(len)
    train[count_feats[i]] = train[cols_to_use[i]].apply(str).apply(lambda x: len(x.split(' ')))

train['keywords_len'] = train['keywords'].apply(str).apply(len)
train['keywords_count'] = train['keywords'].apply(str).apply(lambda x: len(x.split('-')))

for i in np.arange(2):
    test[len_feats[i]] = test[cols_to_use[i]].apply(str).apply(len)
    test[count_feats[i]] = test[cols_to_use[i]].apply(str).apply(lambda x: len(x.split(' ')))
    
test['keywords_len'] = test['keywords'].apply(str).apply(len)
test['keywords_count'] = test['keywords'].apply(str).apply(lambda x: len(x.split('-')))


# In[921]:

train['keywords_count_norm']=train['keywords_count']-train['keywords_count'].median()

train['name_count_norm']=train['name_count']-train['name_count'].median()

train['desc_count_norm']=train['desc_count']-train['desc_count'].median()

test['keywords_count_norm']=test['keywords_count']-test['keywords_count'].median()

test['name_count_norm']=test['name_count']-test['name_count'].median()

test['desc_count_norm']=test['desc_count']-test['desc_count'].median()

In [7]:
# In[922]:

train['title_word_len']=train[['name_len','name_count']].apply(lambda k: (k[0]/(k[1]+1)),axis=1)

test['title_word_len']=test[['name_len','name_count']].apply(lambda k: (k[0]/(k[1]+1)),axis=1)


# In[923]:

train['keyword_word_len']=train[['keywords_len','keywords_count']].apply(lambda k: (k[0]/(k[1]+1)),axis=1)

test['keyword_word_len']=test[['keywords_len','keywords_count']].apply(lambda k: (k[0]/(k[1]+1)),axis=1)


# In[924]:

train['desc_word_len']=train[['desc_len','desc_count']].apply(lambda k: (k[0]/(k[1]+1)),axis=1)

test['desc_word_len']=test[['desc_len','desc_count']].apply(lambda k: (k[0]/(k[1]+1)),axis=1)


# In[925]:

from sklearn.preprocessing import LabelEncoder

feat = ['disable_communication','country','currency']
for x in feat:
    
    le = LabelEncoder()
    le.fit(list(train[x].values) + list(test[x].values))
    train[x] = le.transform(list(train[x]))
    test[x] = le.transform(list(test[x].values))
    


# In[926]:

In [8]:
train['goal_unnormalized'] = train[['goal','currency']].apply(lambda x: c.convert(x[0],le.inverse_transform(int(x[1])),'USD'),axis=1)
test['goal_unnormalized'] = test[['goal','currency']].apply(lambda x: c.convert(x[0],le.inverse_transform(int(x[1])),'USD'),axis=1)

train['goal_norm']=train['goal_unnormalized'] -train['goal_unnormalized'].median()
test['goal_norm']=test['goal_unnormalized'] -test['goal_unnormalized'].median()

train['goal_per_day']=train[['goal','time_project']].apply(lambda x: (x[0]/(x[1]+1))*1e1,axis=1)

test['goal_per_day']=test[['goal','time_project']].apply(lambda x: (x[0]/(x[1]+1))*1e1,axis=1)


# In[959]:

In [9]:
train['goalper_descriptionlength']=train[['goal','desc_count']].apply(lambda x: (x[0]/(x[1]+2)),axis=1)

test['goalper_descriptionlength']=test[['goal','desc_count']].apply(lambda x: (x[0]/(x[1]+2)),axis=1)


# In[928]:

In [10]:
train['state_change_norm']=train[['time_deadline_change','time_project']].apply(lambda x: (x[0]/(x[1]+1))*1e4,axis=1)
test['state_change_norm']=test[['time_deadline_change','time_project']].apply(lambda x: (x[0]/(x[1]+1))*1e4,axis=1)


train['is_12_currency']=train['currency'].apply(lambda x: int(x==12))
test['is_12_currency']=test['currency'].apply(lambda x: int(x==12))

##  </font='black'>We extract text features after cleaning out the stopwords from the data. We use normalized count features of the description length. </font>

In [11]:
import re

# creating a full list of descriptions from train and etst
kickdesc = pd.Series(train['desc'].tolist() + test['desc'].tolist()).astype(str)

# this function cleans punctuations, digits and irregular tabs. Then converts the sentences to lower
def desc_clean(word):
    p1 = re.sub(pattern='(\W+)|(\d+)|(\s+)',repl=' ',string=word)
    p1 = p1.lower()
    return p1

kickdesc = kickdesc.map(desc_clean)

stop = set(stopwords.words('english'))
kickdesc = [[x for x in x.split() if x not in stop] for x in kickdesc]

stemmer = SnowballStemmer(language='english')
kickdesc = [[stemmer.stem(x) for x in x] for x in kickdesc]

kickdesc = [[x for x in x if len(x) > 2] for x in kickdesc]

kickdesc = [' '.join(x) for x in kickdesc]

In [12]:
cv = TfidfVectorizer(max_features=375)

alldesc = cv.fit_transform(kickdesc).todense()

#create a data frame
combine = pd.DataFrame(alldesc)
combine.rename(columns= lambda x: 'descvariable_'+ str(x), inplace=True)

#split the text features

train_text = combine[:train.shape[0]]
test_text = combine[train.shape[0]:]

test_text.reset_index(drop=True,inplace=True)


#test_text.reset_index(drop=True,inplace=True)

## </font='black'> We select the subset of features needed for training data </font>

In [13]:
cols_to_use = ['name_count_norm'
                 ,'desc_count_norm'
                 ,'time_project'
                ,'time_deadline_change'
                  ,'time_launched_change'
               ,'disable_communication'
               ,'goal_per_day'
               ,'is_12_currency'
               ,'cldiff'
               ,'title_word_len'
               ,'keyword_word_len'
               ,'desc_word_len',
               'goalper_descriptionlength'
              ]

target = train['final_status']

# data for modeling
k_train = train[cols_to_use]
k_test=test[cols_to_use]


# In[943]:

In [14]:
X_train = pd.concat([k_train,train_text],axis=1)
X_test=pd.concat([k_test,test_text],axis=1)
print(X_test.shape)

(63465, 388)


In [15]:
from sklearn.cross_validation import train_test_split

## </font='black'> Training Data and Validation Data split </font>

In [16]:
X_tr,X_val,y_tr,y_val=train_test_split(X_train,target,test_size=0.2,random_state=1)

In [25]:


dTrain = xgb.DMatrix(X_train,target)
#dVal=   xgb.DMatrix(X_val,y_val)
dTest  = xgb.DMatrix(X_test)

xgb_params = {
   'objective': 'binary:logistic',
    'booster': 'gbtree',
    'eval_metric': 'logloss',
    'eta': 0.025, 
    'max_depth': 7,
    'lambda': 2,
    'alpha': 0.02,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 1,
    'silent': 1}


## </font='black'> Let us train a XGBoost Model using our features </font>

In [26]:
bst_train=xgb.train(xgb_params, dTrain, 1300,  [(dTrain,'train')],
               verbose_eval=50, early_stopping_rounds=40)

[0]	train-logloss:0.68771
Will train until train-logloss hasn't improved in 40 rounds.
[50]	train-logloss:0.574593
[100]	train-logloss:0.548285
[150]	train-logloss:0.53696
[200]	train-logloss:0.52961
[250]	train-logloss:0.523443
[300]	train-logloss:0.518137
[350]	train-logloss:0.513683
[400]	train-logloss:0.509385
[450]	train-logloss:0.505515
[500]	train-logloss:0.502177
[550]	train-logloss:0.498884
[600]	train-logloss:0.495757
[650]	train-logloss:0.492827
[700]	train-logloss:0.489998
[750]	train-logloss:0.487428
[800]	train-logloss:0.484896
[850]	train-logloss:0.482488
[900]	train-logloss:0.480101
[950]	train-logloss:0.477878
[1000]	train-logloss:0.475649
[1050]	train-logloss:0.47348
[1100]	train-logloss:0.471171
[1150]	train-logloss:0.469225
[1200]	train-logloss:0.46729
[1250]	train-logloss:0.465523


In [27]:
pred = bst_train.predict(dTest, ntree_limit=bst_train.best_ntree_limit)

## </font='black'> We now write our predictions to a file and submit </font>

In [29]:
# In[970]:

### write the file
result=[]
for idx,k in enumerate(pred.tolist()):
    if k>0.4:
        result.append(1)
    else:
        result.append(0)
nBsub = pd.DataFrame({'project_id':test['project_id'],'final_status':result})
nBsub = nBsub[['project_id','final_status']]
nBsub.to_csv("nBstarter.csv",index = False) #0.6526